# 데이터 수집

In [ ]:
import pandas as pd

In [ ]:
rock_samples = pd.read_csv('/workspaces/github_codespace_demo/data/rocksamples.csv')    #절대경로
#rock_samples = pd.read_csv('../data/rocksamples.csv')                                  #상대경로(동작 안 됨)
#디렉토리가 한 단계 아래에 있기 때문에 notebook에서 data찾으므로 오류가 남(../../ 필요)

In [ ]:
rock_samples.head()

In [ ]:
rock_samples.tail()

In [ ]:
rock_samples.info()

In [ ]:
rock_samples.shape      #dataframe's row & column

In [ ]:
rock_samples.shape[0]   #dataframe row  

In [ ]:
rock_samples.shape[1]   #dataframe column

In [ ]:
rock_samples.index      #dataframe index

In [ ]:
rock_samples.columns    #dataframe column name

In [ ]:
rock_samples.dtypes     #dataframe data type(each column)

In [ ]:
rock_samples.describe() #dataframe data summary

## 누락값 검사

In [ ]:
rock_samples.isnull()   #step1. find null value in dataframe

In [ ]:
rock_samples.isnull().sum() #step2. sum each column value(false:0, true:1)

# 데이터 가공

In [ ]:
sample_series = rock_samples['Weight (g)'] 
sample_series

In [ ]:
sample_dataframe = rock_samples[['Type', 'Weight (g)']]
sample_dataframe

In [ ]:
# step1. change column value (weinght (g->kg))
rock_samples['Weight (g)'] = sample_series.apply(lambda x : x * 0.001)
rock_samples['Weight (g)']

In [ ]:
# step2. rename column(Weight(g)->Weight(kg))
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)
rock_samples

## mission별 data frame 만들기

In [ ]:
# make empty dataframe and assign to 'missions'
missions = pd.DataFrame()
type(missions)

In [ ]:
# make new column in missions
missions['Mission'] = rock_samples['Mission'].unique()
missions

## mission별 weight 합계: group by

In [ ]:
# divide dataframe(rock_samples) by 'Mission'(column) => groupby()
# sum each mission's weight. => sum()
# return value is [series]
total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
total_weight

In [ ]:
# merge to missions (dataframe - series)
missions = pd.merge(missions, total_weight, on='Mission')
# rename column
missions.rename(columns={'Weight (kg)':'Sample Weight(kg)'}, inplace=True)
missions

## mission간 중량 차: diff(), filna()

In [ ]:
# calculate difference 
missions['Weight diff'] = missions['Sample Weight(kg)'].diff()
missions

In [ ]:
# replace NaN to 0
missions.fillna(value=0, inplace=True)
missions

## add spaceship data

In [ ]:
# 달 탐사선 = lunar module + command module = crewed area

In [ ]:
# add lunar module data
missions['Lunar module(LM)'] = ['Eagle (LM-5)', 'Interpid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass(kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass(kg)'].diff()
missions.fillna(value=0, inplace=True)
missions

In [ ]:
# add command module data
missions['Command module(CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CSM-108)', 'Kitty Hawk (CSM-110)', 'Endeavor (CSM-112)', 'Casper (CSM-113)', 'America (CSM-114)']
missions['CM mass(kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass(kg)'].diff()
missions.fillna(value=0, inplace=True)
missions

In [ ]:
# add crewed area data
missions['Total weight(kg)'] = missions['LM mass(kg)'] + missions['CM mass(kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

In [ ]:
# add payload data (payload=loaded stuffs total weight at spaceship)
saturnVPayload = 43500
# ratio about crewed area by payload
missions['Crewd area : Payload'] = missions['Total weight(kg)'] / saturnVPayload
# ratio about sample by crewed area
missions['Sample : Crewed area'] = missions['Sample Weight(kg)'] / missions['Total weight(kg)']
# ratio about sample by payload
missions['Sample : Payload'] = missions['Sample Weight(kg)'] / saturnVPayload
missions

# 데이터 분석

## atemis_mission dataframe

In [ ]:
artemis_crewdArea = 26520

# input at dictionary
artemis_dic = {'Mission':['artemis1', 'artemis1b', 'artemis2'],
                'Total weight(kg)':[artemis_crewdArea, artemis_crewdArea, artemis_crewdArea],
                'Payload(kg)':[26988, 37965, 42955]}
# make dataframe by dictionary
artemis_mission = pd.DataFrame(artemis_dic)
artemis_mission

## estimate sample weight

In [ ]:
# calculate average each ratio
crewedArea_payload_ratio = missions['Crewd area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()

In [ ]:
# input expected sample weight
artemis_mission['Sample weight from total(kg)'] = artemis_mission['Total weight(kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload(kg)'] = artemis_mission['Payload(kg)'] * sample_payload_ratio
artemis_mission

In [ ]:
# input estimated sample weight
artemis_mission['Estimated sample weight(kg)'] = (artemis_mission['Sample weight from payload(kg)'] + artemis_mission['Sample weight from total(kg)'])/2
artemis_mission

## guess small amount rocks

In [ ]:
# 남은 양을 rock_samples의 Remiaining(kg)에 저장
rock_samples['Remaining(kg)'] = rock_samples['Weight (kg)'] * rock_samples['Pristine (%)'] * 0.01
rock_samples

In [ ]:
# priority of rock samples(with remain amount)
rock_samples.describe() 

In [ ]:
# 0.16kg 이상 && 50% 이하 remain을 low_samples에 할당
# 각 조건에 괄호 필요??
low_samples = rock_samples.loc[(rock_samples['Weight (kg)'] >= 0.16) & (rock_samples['Pristine (%)'] <= 50)]
low_samples

In [ ]:
low_samples.info()

In [ ]:
# count each type of rock samples
low_samples.groupby('Type')['ID'].count()

## guess needed sample(type of rocks)

In [ ]:
# seperate about 'Basalt'&'Breccia'
needed_samples = low_samples.loc[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples

In [ ]:
needed_samples.info()

In [ ]:
# calculate needed amount about rock samples
needed_samples.groupby('Type')['Weight (kg)'].sum()

## guess uncollected rocks (add to needed_samples)

In [ ]:
# sum rock_samples by type (guess uncollected samples)
rock_samples.groupby('Type')['Weight (kg)'].sum()

In [ ]:
# add crustal data at needed_samples
needed_samples = pd.concat([needed_samples, rock_samples.loc[rock_samples['Type'] == 'Crustal']])
needed_samples

In [ ]:
needed_samples.info()

# 시각화 및 탐색

## make dataframe overview (result)

In [ ]:
# make df overview with needed_samples' type(unique)
overview = pd.DataFrame(needed_samples['Type'].unique(), columns=['Type'])
overview

## calculate needed samples weight

In [ ]:
# calculate each rocks weight
needed_samples_weight = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()   # reset_indes: series -> dataframe
needed_samples_weight

In [ ]:
overview = pd.merge(overview, needed_samples_weight, on='Type')
overview

In [ ]:
# rename 'Weight (kg)' to 'Total weight(kg)'
overview.rename(columns={'Weight (kg)':'Total weight(kg)'}, inplace=True)
overview

## calcuate average weight

In [ ]:
# calculate average weight about each samples type
needed_samples_average = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_average

In [ ]:
overview = pd.merge(overview, needed_samples_average, on='Type')
overview.rename(columns={'Weight (kg)':'Average weight(kg)'}, inplace=True)
overview

## count amount of each samples

In [ ]:
# count each samples by type
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
total_rock_count.rename(columns={'ID':'Num of rocks'}, inplace=True)
total_rock_count

In [ ]:
# merge to overview
overview = pd.merge(overview, total_rock_count, on='Type')
overview

## calculate ratio about rocks

In [ ]:
# total amount of needed rocks
total_rocks = overview['Num of rocks'].sum()
total_rocks

In [ ]:
overview['Percentage of rocks'] = overview['Num of rocks'] / total_rocks
overview

## estimate weight to collect

In [ ]:
# calculate average of estimated sample weight about artemis mission
artemis_average = artemis_mission['Estimated sample weight(kg)'].mean()
artemis_average

In [ ]:
# estimate weight to collect
overview['Weight to collect(kg)'] = artemis_average * overview['Percentage of rocks']
overview

## estimate amount rocks to collect

In [ ]:
# estimate amount to collect
overview['Rocks to collect'] = overview['Weight to collect(kg)'] / overview['Average weight(kg)']
overview